In [13]:
import pandas as pd
import requests
import json

In [74]:
model = '/models/Qwen3-30B-A3B-Instruct-2507-FP8'
url = 'https://10.44.123.3/qwen30-fp8/v1/chat/completions'

def call_llm(model, url, sys_prompt, user_prompt):
    payload = {
                "model": model,
                "messages": [
                    {"role": "system", "content": sys_prompt},
                    {"role": "user", "content": user_prompt},
                ],
                "temperature": 0.1,
                "stream": False,
                "chat_template_kwargs": {"enable_thinking": False},
            }
    url = url
    response = requests.post(url=url, json=payload, verify=False, timeout=30).json()
    resp = response['choices'][0]['message']['content']
    return resp

In [14]:
with open('1_VOP.md', encoding='utf8') as file:
    text = file.read()

In [ ]:
# парсим файл с вопросно-ответными парами
lines = text.split('\n')

text_data = []

current_question = None
current_answer = []

for line in lines:
    if line.startswith('## '):
        if current_question is not None:
            text_data.append([current_question, ' '.join(current_answer)])
            current_answer = []
        current_question = line[3:].strip()
    else:
        current_answer.append(line.strip())

if current_question is not None:
    text_data.append([current_question, ' '.join(current_answer)])
text_data
# df = pd.DataFrame(text_data[:10], columns=['Вопрос', 'Ответ'])
# df

In [ ]:
# делаем из ВОП словари с дубликатами ответов и уникальными ответами
from collections import defaultdict

def group_by_identical_answers(text_data: list):
    """
    На входе список списков, каждый маленький список - вопрос + ответ
    На выходе:
        duplicates: dict {answer: [question1, question2, ...]} для ответов, которые соответствуют >1 вопросу
        unique: список [[question, answer]] для уникальных ответов
    """
    def normalize_answer(answer):
        return ' '.join(answer.strip().split())
    
    answer_groups = defaultdict(list)
    
    for question, answer in text_data:
        norm_answer = normalize_answer(answer)
        answer_groups[norm_answer].append({
            'question': question,
            'original_answer': answer  
        })
    
    # Разделение на группы
    duplicate_qa_groups = []
    unique_qa_pairs = []
    
    for norm_answer, items in answer_groups.items():
        if len(items) > 1:
            duplicate_qa_groups.append({
                'answer': norm_answer,
                # 'original_answer': items[0]['original_answer'],  # Берём первый оригинальный ответ
                'questions': [item['question'] for item in items],
                # 'count': len(items),
                # 'raw_items': items  # Все оригинальные данные
            })
        else:
            unique_qa_pairs.append([items[0]['question'], items[0]['original_answer']])
    
    return duplicate_qa_groups, unique_qa_pairs

# Использование
duplicate_groups, unique_pairs = group_by_identical_answers(text_data)

# Статистика
print(f"Всего пар: {len(text_data)}")
print(f"Уникальных ответов: {len(unique_pairs)}")
print(f"Групп с одинаковыми ответами: {len(duplicate_groups)}")


Всего пар: 843
Уникальных ответов: 416
Групп с одинаковыми ответами: 60


## Работа с дубликатами

In [51]:
duplicate_groups

[{'answer': 'Добрый день! Информируем, что за изменение ВРИ на виды предусматривающие жилищное строительство взимается плата, размер которой определяется в соответствии с Постановлением Правительства Московской области от 31.12.2013 №1190/57. Первоначально необходимо обратиться за услугой "Предоставление расчета размера платы за изменение ВРИ земельного участка". Ссылка на услугу – "https://uslugi.mosreg.ru/services/21663" . Далее, после оплаты Вам необходимо обратиться в Управление Федеральной службы государственной регистрации, кадастра и картографии по Московской области (Росреестр), посредством обращения в любой центр государственных и муниципальных услуг "Мои документы" на территории Московской области за изменением вида разрешенного использования земельного участка.',
  'questions': ['Добрый день! Приводим в соответствие сведения о ВРИ на з/у общего пользования в СНТ(к.н 50:26:0180417:101) Просим пояснить: будет ли начислена плата как за изменение ВРИ при обращении за государстве

In [62]:
sys_prompt = '''
Ты получаешь на вход группу вопросов с одинаковым ответом.

Структура входных данных:
{
  "answer": "универсальный ответ на все вопросы",
  "questions": ["список вопросов"]
}

Твоя задача:
ШАГ 1: Определи общую тему всех вопросов (максимум 3 слова)
ШАГ 2: Сгруппируй вопросы по подтемам. Для каждой группы выдели:
- Название подтемы
- Количество вопросов
- Что объединяет эти вопросы
ШАГ 3: Создай эталонные вопросы для каждой подтемы на основе исходных вопросов (до 3 вопросов на подтему)

ФОРМАТ ОТВЕТА - ТОЛЬКО чистый JSON:
{
  "analysis": {
    "general_topic": "общая тема всех вопросов (1-2 слова)",
    "answer": "оригинальный универсальный ответ"
  },
  "themes": [
    {
      "theme_name": "Название тематики",
      "question_count": число,
      "description": "Краткое описание (что объединяет эти вопросы)",
      "canonical_questions": ["созданные эталонные вопросы (1-5 штук)"],
    }
  ],
}

ВАЖНЫЕ ПРАВИЛА:
1. Ответ ТОЛЬКО в этом JSON-формате, без пояснений
2. canonical_questions должны быть четкими, без конкретных имён/дат/адресов
3. Если вопросы очень похожи, можно оставить 1-2 эталонных
4. Общая тема (general_topic) должна быть краткой
'''

In [63]:
# анализируем дубликаты
duplicate_lst = []
for elem in duplicate_groups:
    user_prompt = f'Группа вопросов: {elem}'
    resp = call_llm(model, url, sys_prompt, user_prompt)
    duplicate_lst.append(resp)
duplicate_lst

c:\Projects\MIO\.venv\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host '10.44.123.3'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
c:\Projects\MIO\.venv\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host '10.44.123.3'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
c:\Projects\MIO\.venv\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host '10.44.123.3'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
c:\Projects\MIO\.venv\Lib\site-packages\urllib3\connectionpool.py:1097: Insecur

['{\n  "analysis": {\n    "general_topic": "изменение ВРИ",\n    "answer": "Добрый день! Информируем, что за изменение ВРИ на виды предусматривающие жилищное строительство взимается плата, размер которой определяется в соответствии с Постановлением Правительства Московской области от 31.12.2013 №1190/57. Первоначально необходимо обратиться за услугой \\"Предоставление расчета размера платы за изменение ВРИ земельного участка\\". Ссылка на услугу – \\"https://uslugi.mosreg.ru/services/21663\\" . Далее, после оплаты Вам необходимо обратиться в Управление Федеральной службы государственной регистрации, кадастра и картографии по Московской области (Росреестр), посредством обращения в любой центр государственных и муниципальных услуг \\"Мои документы\\" на территории Московской области за изменением вида разрешенного использования земельного участка."\n  },\n  "themes": [\n    {\n      "theme_name": "Процедура изменения ВРИ",\n      "question_count": 28,\n      "description": "Вопросы, каса

In [64]:
# придаем ответу вид JSONа
import re
from typing import Any, Optional

def clean_and_parse_json(json_like_string: str) -> Optional[Any]:
    """
    Очищает и парсит строку, похожую на JSON, даже с проблемами.
    Обрабатывает:
    - Строки внутри списков Python ['{...}']
    - Проблемы с экранированием кавычек
    - Лишние символы
    """
    if not json_like_string:
        return None
    
    # Если это список Python, извлекаем первый элемент
    if json_like_string.startswith('[') and json_like_string.endswith(']'):
        try:
            # Безопасно извлекаем содержимое
            match = re.search(r'\[(.*)\]', json_like_string, re.DOTALL)
            if match:
                json_like_string = match.group(1).strip()
        except:
            pass
    
    # Убираем возможные лишние кавычки в начале/конце
    json_like_string = json_like_string.strip()
    if json_like_string.startswith("'") and json_like_string.endswith("'"):
        json_like_string = json_like_string[1:-1]
    elif json_like_string.startswith('"') and json_like_string.endswith('"'):
        json_like_string = json_like_string[1:-1]
    
    # Исправляем двойное экранирование (\\")
    # Заменяем \\" на \"
    json_like_string = re.sub(r'\\\\"', r'\\"', json_like_string)
    
    # Убираем лишние обратные слэши перед кавычками
    json_like_string = re.sub(r'\\+(")', r'\\\1', json_like_string)
    
    # Пробуем распарсить
    try:
        return json.loads(json_like_string)
    except json.JSONDecodeError as e:
        # Пробуем исправить наиболее частые ошибки
        print(f"Первая попытка не удалась: {e}")
        
        # Исправляем незакрытые кавычки в длинных строках
        # Находим все строки в JSON и проверяем их
        try:
            # Убираем проблемные символы
            json_like_string = re.sub(r'([^\\])"([^\\])"', r'\1"\2"', json_like_string)
            
            # Пробуем снова
            return json.loads(json_like_string)
        except json.JSONDecodeError:
            # Последняя попытка: находим JSON объект в строке
            match = re.search(r'\{.*\}', json_like_string, re.DOTALL)
            if match:
                try:
                    return json.loads(match.group())
                except:
                    pass
    
    return None

cleaned_duplicate_list = []
for elem in duplicate_lst:
    cleaned_json = clean_and_parse_json(elem)
    cleaned_duplicate_list.append(cleaned_json)

cleaned_duplicate_list

Первая попытка не удалась: Expecting ',' delimiter: line 4 column 666 (char 740)
Первая попытка не удалась: Expecting ',' delimiter: line 4 column 401 (char 475)
Первая попытка не удалась: Expecting ',' delimiter: line 4 column 269 (char 352)


[{'analysis': {'general_topic': 'изменение ВРИ',
   'answer': 'Добрый день! Информируем, что за изменение ВРИ на виды предусматривающие жилищное строительство взимается плата, размер которой определяется в соответствии с Постановлением Правительства Московской области от 31.12.2013 №1190/57. Первоначально необходимо обратиться за услугой "Предоставление расчета размера платы за изменение ВРИ земельного участка". Ссылка на услугу – "https://uslugi.mosreg.ru/services/21663" . Далее, после оплаты Вам необходимо обратиться в Управление Федеральной службы государственной регистрации, кадастра и картографии по Московской области (Росреестр), посредством обращения в любой центр государственных и муниципальных услуг "Мои документы" на территории Московской области за изменением вида разрешенного использования земельного участка.'},
  'themes': [{'theme_name': 'Процедура изменения ВРИ',
    'question_count': 28,
    'description': 'Вопросы, касающиеся последовательности действий, порядка подачи

In [ ]:
# lst = []
# for _, row in df.iterrows():
#     sys_prompt = '''
#     Ты получаешь на вход строку pandas датафрейма. 
#     Строка в первом столбце датафрейма - вопрос, который задает пользователь.
#     Строка во втором столбце датафрейма - ответ, который дается на вопрос пользователя.
#     Твоя задача - проанализировать вопрос и данный на него ответ и оценить его по критериям релевантности и полноты.
#     Если ответ релевантный и полный, оставь его в неизменном виде.
#     Если ответ нерелевантный, перепиши его, чтобы он лучше соответствовал вопросу. Можешь использовать свои знания.
#     Если ответ повторяется, посмотри на удаляемые вопросы. Составь один общий вопрос, для которого подходит повторяющийся ответ. Остальные удали.
#     '''
#     user_prompt = f'Строка датафрейма: {row}'
#     resp = call_llm(model=model, url=url, sys_prompt=sys_prompt, user_prompt=user_prompt)
#     lst.append(resp)

# lst

c:\Projects\MIO\.venv\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host '10.44.123.3'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
c:\Projects\MIO\.venv\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host '10.44.123.3'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
c:\Projects\MIO\.venv\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host '10.44.123.3'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
c:\Projects\MIO\.venv\Lib\site-packages\urllib3\connectionpool.py:1097: Insecur

## Работа с уникальными парами

In [71]:
unique_pairs

[['Добрый день При заполнении услуги на последнем шаге требуется СОГЛАСИЕ СОБСТВЕННИКА ЗЕМЕЛЬНОГО УЧАСТКА НА УСТАНОВЛЕНИЕ СООТВЕТСТВИЯ ВИДА РАЗРЕШЕННОГО ИСПОЛЬЗОВАНИЯ ЗЕМЕЛЬНОГО УЧАСТКА, без которого невозможно перейти на последний шаг НО! В описании услуги в вопросах-ответах есть требование к документам: Согласие собственника(-ов) земельного участка на установление соответствия вида разрешенного использования земельного участка (в случае, если с заявлением обращается правообладатель, не являющийся собственником этого земельного участка); Я являюсь собственником (не представителем) участка Мне необходимо нотариально заверять заявление или возможно обойтись без этого или возможно ли сделать этот пункт при заполнении необязательным? Спасибо',
  'Добрый день! Если у вас высвечивается данный документ как обязательный, скорее всего вы ответили "да" на вопрос "ЗЕМЕЛЬНЫЙ УЧАСТОК НАХОДИТСЯ У ЗАЯВИТЕЛЯ В АРЕНДЕ/БЕЗВОЗМЕЗДНОМ ПОЛЬЗОВАНИИ/ПОСТОЯННОМ (БЕССРОЧНОМ) ПОЛЬЗОВАНИИ?" или "ЗЕМЕЛЬНЫЙ УЧАСТ

In [78]:
sys_prompt_unique = '''
Ты анализируешь ответы, данные на пользовательские вопросы.

ИНСТРУКЦИЯ:
- Оцени пару "вопрос-ответ" по критериям ниже
- Прими решение: оставить ответ, улучшить ответ, переписав его, или удалить ответ

КРИТЕРИИ ОЦЕНКИ:
- Полнота: отвечает ли на вопрос полностью?
- Уместность: логично ли сочетание вопроса и ответа?

ФОРМАТ ОТВЕТА:
{
    'query': 'текст вопроса',
    'original_answer': 'текст исходного ответа',
    'assessment': 'оставить, переписать или удалить'
    'corrected_answer': 'переписанный ответ или текст исходного ответа, если решено оставить как есть или удалить'
}
'''

unique_pairs_lst = []
for elem in unique_pairs:
    query = elem[0]
    answer = elem[1]
    user_prompt_unique = f'Вопрос пользователя: {query}. Ответ: {answer}'
    res = call_llm(model, url, sys_prompt=sys_prompt_unique, user_prompt=user_prompt_unique)
    unique_pairs_lst.append(res)
    print(len(unique_pairs_lst))

c:\Projects\MIO\.venv\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host '10.44.123.3'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


1


c:\Projects\MIO\.venv\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host '10.44.123.3'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


2


c:\Projects\MIO\.venv\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host '10.44.123.3'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


...

401


c:\Projects\MIO\.venv\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host '10.44.123.3'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


402


c:\Projects\MIO\.venv\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host '10.44.123.3'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


ReadTimeout: HTTPSConnectionPool(host='10.44.123.3', port=443): Read timed out. (read timeout=30)

In [79]:
cleaned_unique_list = []
for elem in unique_pairs_lst:
    cleaned_unique_json = clean_and_parse_json(elem)
    cleaned_unique_list.append(cleaned_unique_json)

cleaned_unique_list

Первая попытка не удалась: Invalid control character at: line 5 column 387 (char 1309)
Первая попытка не удалась: Invalid control character at: line 5 column 59 (char 1382)
Первая попытка не удалась: Invalid control character at: line 5 column 494 (char 1021)
Первая попытка не удалась: Invalid control character at: line 5 column 373 (char 854)
Первая попытка не удалась: Expecting ',' delimiter: line 5 column 1083 (char 1842)
Первая попытка не удалась: Expecting ',' delimiter: line 5 column 1518 (char 3014)
Первая попытка не удалась: Invalid control character at: line 5 column 360 (char 1394)


[{'query': 'Добрый день При заполнении услуги на последнем шаге требуется СОГЛАСИЕ СОБСТВЕННИКА ЗЕМЕЛЬНОГО УЧАСТКА НА УСТАНОВЛЕНИЕ СООТВЕТСТВИЯ ВИДА РАЗРЕШЕННОГО ИСПОЛЬЗОВАНИЯ ЗЕМЕЛЬНОГО УЧАСТКА, без которого невозможно перейти на последний шаг НО! В описании услуги в вопросах-ответах есть требование к документам: Согласие собственника(-ов) земельного участка на установление соответствия вида разрешенного использования земельного участка (в случае, если с заявлением обращается правообладатель, не являющийся собственником этого земельного участка); Я являюсь собственником (не представителем) участка Мне необходимо нотариально заверять заявление или возможно обойтись без этого или возможно ли сделать этот пункт при заполнении необязательным? Спасибо.',
  'original_answer': 'Добрый день! Если у вас высвечивается данный документ как обязательный, скорее всего вы ответили "да" на вопрос "ЗЕМЕЛЬНЫЙ УЧАСТОК НАХОДИТСЯ У ЗАЯВИТЕЛЯ В АРЕНДЕ/БЕЗВОЗМЕЗДНОМ ПОЛЬЗОВАНИИ/ПОСТОЯННОМ (БЕССРОЧНОМ) ПОЛЬЗ